In [1]:
#ONLY RUN THIS FILE ONCE! get_all_objects() is slow... luckily I've saved the resulting dataframe as a .pkl

import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import nltk
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
stemmer = ps()
from nltk.stem.cistem import Cistem
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

#display max rows
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
# Function to format the JSON objects correctly
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [59]:
#Function to take JSON objects and turn them into a DF across multiple pages
#This does all the pages of Unknown makers with 25 results per page. NOTE: This will take ~forever to run
#3290

def get_all_objects():
    data_list = []
    for x in range(0, 3290):
        parameters = {"items_per_page": 25,
                      "page": x
                      }
        response = requests.get("https://risdmuseum.org/api/v1/collection", params=parameters).json()
        for item in response:
            data_list.append(item)

    df = pd.DataFrame()
    for each_dict in data_list:
        new_df = pd.json_normalize(each_dict)
        df = df.append(new_df)
    df['makers'] = df['makers'].astype(str)
    filtered_df = df[df['makers'].str.contains("'name': 'Unknown'")]
    return filtered_df

In [60]:
##Getting all Maker Unknown objects from DB
object_df = get_all_objects()

KeyboardInterrupt: 

In [58]:
##caching with pkl so we don't have to retrieve every time
object_df.to_pickle("./unknown_makers_full_object.pkl")
object_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 0 to 0
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 44 non-null     object
 1   collection         44 non-null     object
 2   credit             44 non-null     object
 3   culture            44 non-null     object
 4   dating             44 non-null     object
 5   datingYearFrom     44 non-null     object
 6   datingYearTo       44 non-null     object
 7   description        44 non-null     object
 8   dimensions         44 non-null     object
 9   edition            44 non-null     object
 10  images             44 non-null     object
 11  inscription        44 non-null     object
 12  makers             44 non-null     object
 13  medium             44 non-null     object
 14  mediumTechnique    44 non-null     object
 15  objectNumber       44 non-null     object
 16  onView             44 non-null     bool  
 17  

In [21]:
##A function to get total numbers for a particular individual category, i.e. culture, dating, etc. 
def get_counts(original_df, category_name, new_name):
    if(isinstance(original_df[category_name].iloc[0], list)):
        mega_list = []
        for row in original_df[category_name]:
            item_list = row
            for item in row: 
                mega_list.append(item)
                mega_list.sort()
        new_dict = {}
        for category in mega_list:
            if not category in new_dict.keys():
                new_dict[category] = 1
            else:
                new_dict[category] +=1
        counts_df = pd.DataFrame([new_dict]).melt()

    else:
        counts = original_df.groupby([category_name]).size()
        counts_df = counts.to_frame().reset_index(category_name)
    counts_df.columns = [new_name, 'Object Count']
    sorted_df = counts_df.sort_values('Object Count', ascending = False)
    return sorted_df

In [17]:
##Get counts for the type of object
get_counts(object_df,'type', 'Object Type')

,Object Type,Object Count
7,Fashion,28
17,Works on Paper,21
3,Costume,16
4,Costume Accessories,12
13,Prints,12
15,Textiles,7
12,Photographs,4
8,Manuscripts,3
14,Sculpture,2
11,Paintings,1


In [18]:
#Get counts for each culture
get_counts(object_df,'culture', 'Culture Label')

,Culture Label,Object Count
1,American,24
0,,12
7,French,6
12,Japanese,3
16,Swiss,2
4,Chinese,2
5,Egyptian,2
6,English,2
11,Italian,2
15,Probably,1


In [ ]:
#Get counts for each date: Question for Denise- how best to categorize?
#Regex needed if I wanted to explore this further
get_counts(object_df,'dating', 'Approx. Date')

In [ ]:
#Get counts for medium type
get_counts(object_df,'medium', 'Medium Type')